In [1]:
# Import MatPlotLib
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Import Numpy and Pandas
import numpy as np
import pandas as pd
import requests
from config import g_key

# Import DateTime
import datetime as dt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

# Import Flask
from flask import Flask, jsonify


Bad key "text.kerning_factor" on line 4 in
C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


## SQLAlchemy
Query data from city.sqlite

In [61]:
engine = create_engine("sqlite:///Resources/city.sqlite")
connection = engine.connect()

# Declare bases and map class
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [62]:
# view all of the classes
Base.classes.keys()

# Save references to each table
City = Base.classes.city

In [63]:
session = Session(engine)

    # query results to a dictionary using date as the key and prcp as the value
results = session.query(City.city,
                        City.state,
                        City.country,
                        City.crime_index,
                        City.safety_index,
                        City.quality_of_life_index,
                        City.purchasing_power_index,
                        City.health_care_index,
                        City.cost_of_living_index,
                        City.traffic_commute_time_index,
                        City.price_to_income_ratio,
                        City.affordability_index,
                        City.lat,
                        City.long
                       ).\
    order_by(City.city).all()

session.close()

    # List date and precipitation
data = []
for city, state, country, crime, safety, life, purchasing, health, cost, traffic, price, affordability, lat, long in results:
    data_dict = {}
    data_dict["city"] = city
    data_dict["state"] = state
    data_dict["country"] = country
    data_dict["crime_index"] = crime
    data_dict["safety_index"] = safety
    data_dict["quality_of_life_index"] = life
    data_dict["purchasing_power_index"] = purchasing
    data_dict["health_care_index"] = health
    data_dict["cost_of_living_index"] = cost
    data_dict["traffic_commute_time_index"] = traffic
    data_dict["price_to_income_ratio"] = price
    data_dict["affordability_index"] = affordability
    data_dict["lat"] = lat
    data_dict["long"] = long
    data.append(data_dict)
    
    # Jsonify the data
#return jsonify(data)

In [64]:
print(data)

[{'city': 'Albuquerque', 'state': 'NM', 'country': 'United States', 'crime_index': 68.63, 'safety_index': 31.37, 'quality_of_life_index': 178.03, 'purchasing_power_index': 122.06, 'health_care_index': 72.22, 'cost_of_living_index': 64.2, 'traffic_commute_time_index': 26.44, 'price_to_income_ratio': 3.05, 'affordability_index': 4.33, 'lat': 35.0843859, 'long': -106.650422}, {'city': 'Atlanta', 'state': 'GA', 'country': 'United States', 'crime_index': 63.63, 'safety_index': 36.37, 'quality_of_life_index': 169.9, 'purchasing_power_index': 145.47, 'health_care_index': 64.21, 'cost_of_living_index': 72.26, 'traffic_commute_time_index': 44.22, 'price_to_income_ratio': 2.93, 'affordability_index': 4.6, 'lat': 33.7489954, 'long': -84.3879824}, {'city': 'Austin', 'state': 'TX', 'country': 'United States', 'crime_index': 34.6, 'safety_index': 65.4, 'quality_of_life_index': 192.57, 'purchasing_power_index': 143.76, 'health_care_index': 70.58, 'cost_of_living_index': 69.94, 'traffic_commute_time_i

## Query data from Google API
Get coordinate information for each city

In [53]:
# Get base url for Google API
url = "https://maps.googleapis.com/maps/api/geocode/json"

In [54]:
# Open city.csv and read dataframe
csv_path = "./Resources/city.csv"
pd.read_csv(csv_path)
city_df = pd.read_csv(csv_path).sort_values("city")
city_df.head(10)

,city,state,country,crime_index,safety_index,quality_of_life_index,purchasing_power_index,health_care_index,cost_of_living_index,traffic_commute_time_index,pollution_index,price_to_income_ratio,affordability_index
20,Albuquerque,NM,United States,68.63,31.37,178.03,122.06,72.22,64.20,26.44,22.54,3.05,4.33
26,Atlanta,GA,United States,63.63,36.37,169.90,145.47,64.21,72.26,44.22,44.70,2.93,4.60
4,Austin,TX,United States,34.60,65.40,192.57,143.76,70.58,69.94,35.07,36.97,4.03,3.36
28,Baltimore,MD,United States,71.25,28.75,167.98,125.36,80.29,75.24,32.74,43.88,2.59,5.26
16,Boston,MA,United States,33.21,66.79,181.01,121.89,75.60,82.68,43.36,25.29,8.48,1.59
2,Charlotte,NC,United States,45.17,54.83,195.85,145.53,75.00,75.63,38.02,28.22,2.40,5.58
32,Chicago,IL,United States,64.64,35.36,161.24,133.36,68.37,77.75,40.99,41.03,3.54,3.79
19,Cincinnati,OH,United States,51.07,48.93,178.51,126.99,68.50,63.48,26.61,39.77,2.40,5.70
1,Columbus,OH,United States,42.36,57.64,195.88,127.33,75.28,65.79,25.21,24.37,2.31,5.97
6,Dallas,TX,United States,49.52,50.48,188.66,158.15,66.33,66.95,36.14,39.80,2.36,5.72


In [55]:
# Set only cities column
cities = city_df[["city"]]
cities.head()

,city
20,Albuquerque
26,Atlanta
4,Austin
28,Baltimore
16,Boston


In [18]:
# Add latitude and longitude columns for the cities dataframe
cities['location'] = ""
cities.head()

<ipython-input-18-c604b4d5f095>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities['location'] = ""


,city,location
20,Albuquerque,
26,Atlanta,
4,Austin,
28,Baltimore,
16,Boston,


In [12]:
# Assign parameter for Google API
params = {
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
# for index, row in cities.iterrows():

    # get city type from df
city_type = "Irvine"

    # add address to params dict
params['address'] = city_type

    # assemble url and make API request
print(f"Retrieving Results for Index {index}: {city_type}.")
response = requests.get(url, params=params).json()
    
    # extract results
results = response['results']
results[0]['geometry']['location']
    
cities.loc[index, 'location'] = str(results[0]['geometry']['location'])
cities.head()

Retrieving Results for Index 20: Irvine.


C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-12-dbde712c36c2>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities.loc[index, 'location'] = str(results[0]['geometry']['location'])


,city,location
20,Albuquerque,"{'lat': 33.6845673, 'lng': -117.8265049}"
26,Atlanta,
4,Austin,
28,Baltimore,
16,Boston,


In [19]:
# Assign parameter for Google API
params = {
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in cities.iterrows():

    # get city type from df
    city_type = row['city']

    # add address to params dict
    params['address'] = city_type

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_type}.")
    response = requests.get(url, params=params).json()
    
    # extract results
    results = response['results']
    results
    
    try:
        cities.loc[index, 'location'] = str(results[0]['geometry']['location'])
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 20: Albuquerque.
------------
Retrieving Results for Index 26: Atlanta.


C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-19-1e37cc1ed072>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities.loc[index, 'location'] = str(results[0]['geometry']['location'])


------------
Retrieving Results for Index 4: Austin.
------------
Retrieving Results for Index 28: Baltimore.
------------
Retrieving Results for Index 16: Boston.
------------
Retrieving Results for Index 2: Charlotte.
------------
Retrieving Results for Index 32: Chicago.
------------
Retrieving Results for Index 19: Cincinnati.
------------
Retrieving Results for Index 1: Columbus.
------------
Retrieving Results for Index 6: Dallas.
------------
Retrieving Results for Index 27: Denver.
------------
Retrieving Results for Index 40: Detroit.
------------
Retrieving Results for Index 31: Honolulu.
------------
Retrieving Results for Index 25: Houston.
------------
Retrieving Results for Index 29: Indianapolis.
------------
Retrieving Results for Index 11: Jacksonville.
------------
Retrieving Results for Index 10: Kansas City.
------------
Retrieving Results for Index 35: Las Vegas.
------------
Retrieving Results for Index 38: Los Angeles.
------------
Retrieving Results for Index 0:

In [20]:
cities.head()

,city,location
20,Albuquerque,"{'lat': 35.0843859, 'lng': -106.650422}"
26,Atlanta,"{'lat': 33.7489954, 'lng': -84.3879824}"
4,Austin,"{'lat': 30.267153, 'lng': -97.7430608}"
28,Baltimore,"{'lat': 39.2903848, 'lng': -76.6121893}"
16,Boston,"{'lat': 42.3600825, 'lng': -71.0588801}"


In [56]:
# Add latitude and longitude columns for the cities dataframe
cities['lat'] = ""
cities['long'] = ""
cities.head()

<ipython-input-56-b04e51b5b019>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities['lat'] = ""
<ipython-input-56-b04e51b5b019>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities['long'] = ""


,city,lat,long
20,Albuquerque,,
26,Atlanta,,
4,Austin,,
28,Baltimore,,
16,Boston,,


In [57]:
# Assign parameter for Google API
params = {
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in cities.iterrows():

    # get city type from df
    city_type = row['city']

    # add address to params dict
    params['address'] = city_type

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_type}.")
    response = requests.get(url, params=params).json()
    
    # extract results
    results = response['results']
    results
    
    try:
        cities.loc[index, 'lat'] = results[0]['geometry']['location']['lat']
        cities.loc[index, 'long'] = results[0]['geometry']['location']['lng']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 20: Albuquerque.


C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-57-7cd13ff7d429>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities.loc[index, 'lat'] = results[0]['geometry']['location']['lat']
<ipython-input-57-7cd13ff7d429>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cities.loc[index, 'long'] = r

------------
Retrieving Results for Index 26: Atlanta.
------------
Retrieving Results for Index 4: Austin.
------------
Retrieving Results for Index 28: Baltimore.
------------
Retrieving Results for Index 16: Boston.
------------
Retrieving Results for Index 2: Charlotte.
------------
Retrieving Results for Index 32: Chicago.
------------
Retrieving Results for Index 19: Cincinnati.
------------
Retrieving Results for Index 1: Columbus.
------------
Retrieving Results for Index 6: Dallas.
------------
Retrieving Results for Index 27: Denver.
------------
Retrieving Results for Index 40: Detroit.
------------
Retrieving Results for Index 31: Honolulu.
------------
Retrieving Results for Index 25: Houston.
------------
Retrieving Results for Index 29: Indianapolis.
------------
Retrieving Results for Index 11: Jacksonville.
------------
Retrieving Results for Index 10: Kansas City.
------------
Retrieving Results for Index 35: Las Vegas.
------------
Retrieving Results for Index 38: Lo

In [58]:
# Check results
cities

,city,lat,long
20,Albuquerque,35.0844,-106.65
26,Atlanta,33.749,-84.388
4,Austin,30.2672,-97.7431
28,Baltimore,39.2904,-76.6122
16,Boston,42.3601,-71.0589
2,Charlotte,35.2271,-80.8431
32,Chicago,41.8781,-87.6298
19,Cincinnati,39.1031,-84.512
1,Columbus,39.9612,-82.9988
6,Dallas,32.7767,-96.797


In [47]:
lat = cities["lat"]
long = cities["long"]
location = f'[{lat}, {long}]'
location

'[20    35.0844\n26     33.749\n4     30.2672\n28    39.2904\n16    42.3601\n2     35.2271\n32    41.8781\n19    39.1031\n1     39.9612\n6     32.7767\n27    39.7392\n40    42.3314\n31    21.3069\n25    29.7604\n29    39.7684\n11    30.3322\n10    39.0997\n35    36.1699\n38    34.0522\n0     43.0731\n37    25.7617\n34    43.0389\n15    44.9778\n39    40.7128\n14    37.8044\n23    28.5383\n36    39.9526\n33    33.4484\n24    40.4406\n8     45.5051\n12    38.5816\n17     38.627\n18    40.7608\n9     29.4241\n3     32.7157\n21    37.7749\n5     37.3382\n7     47.6062\n13    27.9506\n22    32.2226\n30    47.7511\nName: lat, dtype: object, 20    -106.65\n26    -84.388\n4    -97.7431\n28   -76.6122\n16   -71.0589\n2    -80.8431\n32   -87.6298\n19    -84.512\n1    -82.9988\n6     -96.797\n27    -104.99\n40   -83.0458\n31   -157.858\n25   -95.3698\n29   -86.1581\n11   -81.6557\n10   -94.5786\n35    -115.14\n38   -118.244\n0    -89.4012\n37   -80.1918\n34   -87.9065\n15    -93.265\n39    -74.00

In [52]:
new_df = pd.DataFrame({"name": cities["city"],
                       "location": '[f{lat}, f{long}]'
                      })
new_df

,name,location
20,Albuquerque,"[f{lat}, f{long}]"
26,Atlanta,"[f{lat}, f{long}]"
4,Austin,"[f{lat}, f{long}]"
28,Baltimore,"[f{lat}, f{long}]"
16,Boston,"[f{lat}, f{long}]"
2,Charlotte,"[f{lat}, f{long}]"
32,Chicago,"[f{lat}, f{long}]"
19,Cincinnati,"[f{lat}, f{long}]"
1,Columbus,"[f{lat}, f{long}]"
6,Dallas,"[f{lat}, f{long}]"


In [60]:
# Export to csv file
export_csv = cities.to_csv (r'Resources/location.csv', index = None, header=True)